In [2]:
VERSION

v"1.8.2"

In [3]:
ENV["COLUMNS"] = 1000
ENV["LINES"] = 20;

In [4]:
# using Pkg
# Pkg.add("GLMakie")
# Pkg.update("GLMakie")
# Pkg.build("GLMakie")

In [31]:
using DataFrames
using GLMakie

using Revise
using Astroshaper
import SPICE

In [169]:
meta_kernel = "/Users/masanorikanamaru/Dropbox/spice/hera/kernels/mk/hera_study_PO_EMA_2024_shapeViewer.tm"

SPICE.furnsh(meta_kernel)

In [170]:
et_start = SPICE.utc2et("2027-02-09T22:00:00")
et_end   = SPICE.utc2et("2027-02-11T22:00:00")
step     = 3600/20

et_range = et_start:step:et_end

8.554824691849957e8:180.0:8.556552691849957e8

In [171]:
"""
Obtain a dataframe of ephemerides 

# Arguments
- `targ`   : Target body name
- `ets`    : AbstractVetor of observer epoch
- `ref`    : Reference frame of output position vector
- `abcorr` : Aberration correction flag
- `obs`    : Observing body name
"""
function spkpos_df(targ, ets::AbstractVector, ref, abcorr, obs)
    df = DataFrame(et=Float64[], x=Float64[], y=Float64[], z=Float64[], lt=Float64[])
    for et in ets
        pos, lt = SPICE.spkpos(targ, et, ref, abcorr, obs)  # pos [km], lt [s]
        pos = SPICE.convrt.(pos, "km", "m")
        push!(df, (et, pos..., lt))
    end
    df
end

function spkpos_vec(targ, ets::AbstractVector, ref, abcorr, obs)
    positions = Vector{Float64}[]
    for et in ets
        pos, lt = SPICE.spkpos(targ, et, ref, abcorr, obs)  # pos [km], lt [s]
        pos = SPICE.convrt.(pos, "km", "m")
        push!(positions, collect(pos))
    end
    positions
end


"""
# Arguments
- `from` : Name of the frame to transform from
- `to`   : Name of the frame to transform to
- `ets`  : Epoch of the rotation matrix

# Return
- `rotate` : A rotation matrix
"""
pxform(from, to, ets) = [SPICE.pxform(from, to, et) for et in ets];

In [172]:
sun_d1 = spkpos_vec("SUN",       et_range, "DIDYMOS_FIXED",   "None", "DIDYMOS")
sun_d2 = spkpos_vec("SUN",       et_range, "DIMORPHOS_FIXED", "None", "DIMORPHOS")
d1_d2  = spkpos_vec("DIDYMOS",   et_range, "DIMORPHOS_FIXED", "None", "DIMORPHOS")
d2_d1  = spkpos_vec("DIMORPHOS", et_range, "DIDYMOS_FIXED",   "None", "DIDYMOS");

In [173]:
D1_TO_D2 = pxform("DIDYMOS_FIXED",   "DIMORPHOS_FIXED", et_range)
D2_TO_D1 = pxform("DIMORPHOS_FIXED", "DIDYMOS_FIXED",   et_range);

In [174]:
SPICE.kclear()

In [183]:
fig = Figure()
ax = Axis3(fig[1, 1], aspect=:data)

# scatter!(0,0,0)
# scatter!(sun_d1.x, sun_d1.y, sun_d1.z, color=:orange)
# scatter!(d2_d1.x, d2_d1.y, d2_d1.z, color=:skyblue)
# scatter!(sun_d1.x, sun_d1.y, sun_d1.z, color=:orange)
# scatter!(d2_d1.x, d2_d1.y, d2_d1.z, color=:skyblue)

# scatter!([r[1] for r in sun_d1], [r[2] for r in sun_d1], [r[3] for r in sun_d1], color=:orange)

display(fig)

GLMakie.Screen(...)

- まずはDidymosだけで熱計算してみる
- まずはDimorphosだけで熱計算してみる
- 二重小惑星として熱計算してみる

## 66391 Moshup (1999 KW4)

- [JPL database](https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=1999KW4&view=VOSP)

In [34]:
params_orbit = Dict()

params_orbit[:a] = 0.6423718937095335  # semi-mojor axis [AU]
params_orbit[:e] = 0.6883626074960387  # eccentricity
params_orbit[:I] = 38.87892427076324   # inclination [deg]
params_orbit[:Ω] = 244.8999897709153   # longitude of the ascending node [deg]
params_orbit[:ω] = 192.6452855048569   # argument of periapsis [deg]
params_orbit[:Φ] = 182.5701683309563   # mean anomaly [deg]

params_orbit[:μ] = GM☉

orbit1 = OrbitalElements(params_orbit)

--------------------
  Orbital elements  
--------------------
    Semi-mojor axis         : a  = 0.6423718937095335 [AU]
    Eccentricity            : e  = 0.6883626074960387 [-]
    Lon. of ascending node  : Ω  = 244.89998977091534 [deg]
    Argument of periapsis   : ω  = 192.6452855048569 [deg]
    Inclination             : I  = 38.87892427076324 [deg]
    Periapsis passage time  : tₚ = -8.239849462571867e6 [sec]
    Mean anomaly            : Φ  = 182.5701683309563 [deg]
--------------------
  Other parameters  
--------------------
    Gravitational parameter : μ = 1.32712440018e20 [m^3/s^2]
    Mean motion             : n = 1.9143629523143177 [deg/day]
    Orbital period          : P = 188.05211392374034 [day]
------------------
  Time-variables  
------------------
    Time                    : t  = 0.0 [sec]
    Eccentric anomaly       : u  = 181.5223575891701 [deg]
    True anomaly            : ν  = 180.6540778308607 [deg]
    Position                : r  = [-1.62213450924202e1

In [36]:
params_spin = Dict()
params_spin[:λ] = 326.
params_spin[:β] = -65.
params_spin[:P] = 2.7645

spin1 = SpinParams(params_spin, orbit1)

-------------------
  Spin parameters  
-------------------
Right ascension (RA) : α = 22.298395064751112 [deg]
Declination (Dec)    : δ = -67.747791152182 [deg]
Ecliptic longitude   : λ = 326.0 [deg]
Ecliptic latitude    : β = -65.0 [deg]
Obliquity            : ε = 147.3267557672455 [deg]
Spin period          : P = 2.7645000000000004 [hours]
Spin rate            : ω = 0.0006313363183195259 [rad/sec]
Vernal equinox lon.  : γ = 125.15365519198035 [deg]
                           (longitude from the periheion direction)
Time                 : t  = 0.0 [sec]
Initial spin phase   : ϕ₀ = 0.0 [deg]
Spin phase           : ϕ  = 0.0 [deg]


In [37]:
# params_thermo = ParamsThermo(
#     A_B   = 0.04,
#     A_TH  = 0.,
#     k     = 0.1,
#     ρ     = 1270.,
#     Cp    = 600.,
#     ϵ     = 1.,
#     P     = spin1.T,
#     Δt    = spin1.T/72,
#     t_bgn = 0.,
#     # t_end = spin.T*10,
#     t_end = orbit1.T,
#     Δz    = 0.015,
#     z_max = 0.6,
# )

## Satellite Squannit

- [JPL database](https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=1999KW4&view=VOSP)

In [38]:
params_orbit = Dict()

params_orbit[:a] = 2548 / AU  # semi-mojor axis [AU]
params_orbit[:e] = 0.0004     # eccentricity
params_orbit[:I] = 156.1      # inclination [deg]
params_orbit[:Ω] = 105.4      # longitude of the ascending node [deg]
params_orbit[:ω] = 319.7      # argument of periapsis [deg]
params_orbit[:tₚ] = 0.        # periapsis passage time [sec]

m₁ = 2.353e12
m₂ = 0.135e12
params_orbit[:μ] = G * (m₁ + m₂)

orbit2 = OrbitalElements(params_orbit)

--------------------
  Orbital elements  
--------------------
    Semi-mojor axis         : a  = 1.7032328067237854e-8 [AU]
    Eccentricity            : e  = 0.0004 [-]
    Lon. of ascending node  : Ω  = 105.4 [deg]
    Argument of periapsis   : ω  = 319.7 [deg]
    Inclination             : I  = 156.1 [deg]
    Periapsis passage time  : tₚ = 0.0 [sec]
    Mean anomaly            : Φ  = -0.0 [deg]
--------------------
  Other parameters  
--------------------
    Gravitational parameter : μ = 166.051187528 [m^3/s^2]
    Mean motion             : n = 495.97322554307357 [deg/day]
    Orbital period          : P = 0.725845633311783 [day]
------------------
  Time-variables  
------------------
    Time                    : t  = 0.0 [sec]
    Eccentric anomaly       : u  = 0.0 [deg]
    True anomaly            : ν  = 0.0 [deg]
    Position                : r  = [2546.9808000000003, 0.0, 0.0] [m]
    Velocity                : v  = [-0.0, 0.25538477268155224, 0.0] [m/s]
    Solar irradiati

In [42]:
params_spin = Dict()
params_spin[:λ] = 326.
params_spin[:β] = -62.
params_spin[:P] = SPICE.convrt(orbit2.P, "seconds", "hours")

spin2 = SpinParams(params_spin, orbit1)

-------------------
  Spin parameters  
-------------------
Right ascension (RA) : α = 15.829825508029357 [deg]
Declination (Dec)    : δ = -66.1369604954399 [deg]
Ecliptic longitude   : λ = 326.0 [deg]
Ecliptic latitude    : β = -62.0 [deg]
Obliquity            : ε = 147.00101898240996 [deg]
Spin period          : P = 17.42029519948279 [hours]
Spin rate            : ω = 0.00010018941883637818 [rad/sec]
Vernal equinox lon.  : γ = 119.6521850206004 [deg]
                           (longitude from the periheion direction)
Time                 : t  = 0.0 [sec]
Initial spin phase   : ϕ₀ = 0.0 [deg]
Spin phase           : ϕ  = 0.0 [deg]


In [55]:
params_thermo = ThermoParams(
    A_B   = 0.04,
    A_TH  = 0.,
    k     = 0.1,
    ρ     = 1270.,
    Cp    = 600.,
    ϵ     = 1.,
    P     = spin1.P,
    Nt    = 144,
    t_bgn = 0.,
    t_end = spin2.P * 2,
    Nz    = 40,
    z_max = 0.6,
)

Thermophysical parameters
-------------------------
A_B   : 0.04
A_TH  : 0.0
k     : 0.1
ρ     : 1270.0
Cp    : 600.0
ϵ     : 1.0
-------------------------
t_bgn : 0.0
t_bgn : 0.0 (Normalized by period P)
t_end : 125426.1254362761
t_end : 12.602854186639746 (Normalized by period P)
Nt    : 144
Δt    : 877.105772281651
Δt    : 0.0881318474590192 (Normalized by period P)
-------------------------
z_max : 0.6
z_max : 4.683432404907855 (Normalized by skin depth l)
Nz    : 40
Δz    : 0.015384615384615385
Δz    : 0.12008801038225268 (Normalized by skin depth l)
-------------------------
P     : 9952.2
l     : 0.12811116893055807
Γ     : 276.04347483684523
λ     : 0.4863217700643013
-------------------------


In [22]:
# kernels = [
#     "../../spice/kernels/generic_kernels/lsk/naif0012.tls",
#     "../../spice/kernels/generic_kernels/spk/planets/de440.bsp",
#     "../../spice/kernels/generic_kernels/pck/de-403-masses.tpc",
#     "../../spice/kernels/generic_kernels/pck/pck00010.tpc",
# ]
# bodies = ["Sun", "Mercury", "Venus", "Earth_barycenter", "Mars_barycenter"]

# Astroshaper.plot_orbits(kernels, bodies, orbit1)

In [59]:
# draw(shape1, shape2, "tmp.mp4")

In [61]:
tmp = run_binary(shape1, shape2, orbit1, orbit2, spin1, spin2, params_thermo);

LoadError: MethodError: no method matching run_binary_TPM!(::ShapeModel{Int64, Vector{StaticArrays.SVector{3, Float64}}, Vector{StaticArrays.SVector{3, Int64}}, StructArrays.StructVector{Astroshaper.Facet{StaticArrays.SVector{3, Float64}, Float64, StructArrays.StructVector{Astroshaper.VisibleFacet, NamedTuple{(:id, :f, :d, :d̂), Tuple{Vector{Int64}, Vector{Float64}, Vector{Float64}, Vector{StaticArrays.SVector{3, Float64}}}}, Int64}, Astroshaper.Flux{Float64}, Vector{Float64}, Vector{Float64}}, NamedTuple{(:A, :B, :C, :center, :normal, :area, :visiblefacets, :flux, :temps, :_temps_, :force), Tuple{Vector{StaticArrays.SVector{3, Float64}}, Vector{StaticArrays.SVector{3, Float64}}, Vector{StaticArrays.SVector{3, Float64}}, Vector{StaticArrays.SVector{3, Float64}}, Vector{StaticArrays.SVector{3, Float64}}, Vector{Float64}, Vector{StructArrays.StructVector{Astroshaper.VisibleFacet, NamedTuple{(:id, :f, :d, :d̂), Tuple{Vector{Int64}, Vector{Float64}, Vector{Float64}, Vector{StaticArrays.SVe